# Import Libraries

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
#from imblearn.over_sampling import SMOTE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd
import logging

In [3]:
data=pd.read_csv('Preprocessed_data.csv')

#Dividing target variable from the main dataset

X = data.iloc[: , 0:-1]
Y = data.iloc[: , -1] 

In [4]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1)

# Encoding categorical data
## Encoding the Independent Variable

In [5]:
ordinal_encoder = OrdinalEncoder()
X_train_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(X_train.select_dtypes(exclude='number')))
X_train_cat_encoded.columns = X_train.select_dtypes(exclude='number').columns

X_test_cat_encoded = pd.DataFrame(ordinal_encoder.transform(X_test.select_dtypes(exclude='number')))
X_test_cat_encoded.columns = X_test.select_dtypes(exclude='number').columns

## Encoding the Independent Variable

In [6]:
label_encoder = LabelEncoder()
Y_train_cat_encoded= pd.DataFrame(label_encoder.fit_transform(Y_train))
Y_test_cat_encoded = pd.DataFrame(label_encoder.transform(Y_test))

## Standardization

In [7]:
sc = StandardScaler()
X_train_sc=pd.DataFrame(sc.fit_transform(X_train.select_dtypes(exclude='O')))
X_test_sc=pd.DataFrame(sc.transform(X_test.select_dtypes(exclude='O')))

X_train_sc.columns=X_train.select_dtypes(exclude='O').columns
X_test_sc.columns=X_test.select_dtypes(exclude='O').columns

## Combining data

In [8]:
X_train_final=pd.concat([X_train_sc,X_train_cat_encoded],axis=1)
X_test_final=pd.concat([X_test_sc,X_test_cat_encoded],axis=1)

## Handling imbalanced Dataset
### Since the dataset is small, will use over-sampling: SMOTE technique to balance the data

In [12]:
pip install imblearn

  Using cached imbalanced_learn-0.9.1-py3-none-any.whl (199 kB)
  Using cached imbalanced_learn-0.9.0-py3-none-any.whl (199 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
from imblearn.over_sampling import SMOTE

In [15]:
X_train_resample,Y_train_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_train_final,Y_train_cat_encoded)
X_test_resample,Y_test_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_test_final,Y_test_cat_encoded)

X_train_resample.shape,X_test_resample.shape,Y_train_resample.shape,Y_test_resample.shape

((7852, 28), (1476, 28), (7852, 1), (1476, 1))

## Feature Selection

In [16]:
print('Training dataset shape:', X_train_resample.shape, Y_train_resample.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample.shape)

Y_train_resample_flat = Y_train_resample.to_numpy().ravel()
Y_test_resample_flat = Y_test_resample.to_numpy().ravel()

print('Training dataset shape:', X_train_resample.shape, Y_train_resample_flat.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample_flat.shape)

Training dataset shape: (7852, 28) (7852, 1)
Testing dataset shape: (1476, 28) (1476, 1)
Training dataset shape: (7852, 28) (7852,)
Testing dataset shape: (1476, 28) (1476,)


### Forward selection approach

In [17]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
forward_fs = sfs(rf , k_features=10,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)
forward_fs = forward_fs.fit(X_train_resample, Y_train_resample_flat)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:   24.4s finished

[2022-11-25 01:21:17] Features: 1/10 -- score: 0.7929168406670263[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   36.8s finished

[2022-11-25 01:21:54] Features: 2/10 -- score: 0.9681605695589244[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:   34.5s finished

[2022-11-25 01:22:29] Features: 3/10 -- score: 0.9875189238060873[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [43]:
feat_names = list(forward_fs.k_feature_names_)
print(feat_names)
X_train_new=X_train_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication','sick', 'goitre', 'hypopituitary', 'psych', 'TBG_measured']]
X_test_new=X_test_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication','sick', 'goitre', 'hypopituitary', 'psych', 'TBG_measured']]

['TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'sick', 'goitre', 'FTI_measured', 'TBG_measured', 'referral_source']


In [50]:
rf_model=rf.fit(X_train_new,Y_train_resample_flat)
#Checking the metrics of Random Forest
def print_Score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        pred=clf.predict(x_train)
        clf_report=pd.DataFrame(classification_report(y_train,pred,output_dict=True))
        print("Train Result:\n===============")
        print(str(int(accuracy_score(y_train,pred)*100))+"%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("-----------------------------------")
        
       
    elif train==False:
        pred=clf.predict(x_test)
        clf_report=pd.DataFrame(classification_report(y_test,pred,output_dict=True))
        print("Test Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("---------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")
        
        
        
        
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Train Result:
99%
---------------------------------
Classification Report:
                     0            1            2       3  accuracy  \
precision     0.994428     1.000000     0.998474     1.0  0.998217   
recall        1.000000     0.992868     1.000000     1.0  0.998217   
f1-score      0.997206     0.996421     0.999236     1.0  0.998217   
support    1963.000000  1963.000000  1963.000000  1963.0  0.998217   

             macro avg  weighted avg  
precision     0.998225      0.998225  
recall        0.998217      0.998217  
f1-score      0.998216      0.998216  
support    7852.000000   7852.000000  
-----------------------------------
Test Result:
Accuracy Score:97.83%
---------------------------------
Classification Report:
                    0           1           2    3  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.0   0.97832     0.735632   
recall       1.000000    0.991870    0.943089  0.0   0.97832     0.733740   
f1-score     0.97041

C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
##Hyper parameter tuning
RF=RandomForestClassifier()
model=RF.fit(X_train_new,Y_train_resample_flat)

print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

2021-12-29 14:42:24,480 INFO:Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

2021-12-29 14:42:24,580 INFO:Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-scor

Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-score     0.972332    0.997963    0.972860   0.98103     0.981

In [21]:
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
#No of features consider at every split
max_features=['auto','sqrt','log2']
#maximum no of levels in trees
max_depth=[int(x) for x in np.linspace(10,1000,10)]
#minimum no of samples required to split a node
min_samples_split=[1,3,4,5,7,9]
#minimum samples leafs required at each leaf node
min_sample_leafs=[1,2,4,6,8]


random_grid={'n_estimators':n_estimators,
'max_features':max_features,
'max_depth':max_depth,
'min_samples_split':min_samples_split,
'min_samples_leaf':min_sample_leafs,
'criterion':['entropy','gini']}
print(random_grid)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [24]:
rcv=RandomizedSearchCV(estimator=RandomForestClassifier(),param_distributions=random_grid,n_iter=100,cv=3,verbose=2,random_state=0,n_jobs=-1)

rcv.fit(X_train_new,Y_train_resample_flat)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "C:\Users\SOUMALLA-ANTARA\Anaconda3\lib\site-packages\joblib\parallel.py", line 1041, in __call__
    if self.dispatch_one_batch(iterator):
 

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 3, 4, 5, 7, 9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=0, verbose=2)

In [25]:
rcv.best_estimator_


RandomForestClassifier(criterion='entropy', max_depth=230, min_samples_leaf=2,
                       min_samples_split=3, n_estimators=400)

In [32]:
best_random_grid=rcv.best_estimator_

print_Score(best_random_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [37]:
param_grid = {
    'criterion': [rcv.best_params_['criterion']],
    'max_depth': [rcv.best_params_['max_depth']],
    'max_features': [rcv.best_params_['max_features']],
    'min_samples_leaf': [rcv.best_params_['min_samples_leaf'], 
                         rcv.best_params_['min_samples_leaf']+2],
    'min_samples_split': [rcv.best_params_['min_samples_split'] - 1,
                          rcv.best_params_['min_samples_split'], 
                          rcv.best_params_['min_samples_split'] +1],
    'n_estimators': [rcv.best_params_['n_estimators'] - 100, 
                     rcv.best_params_['n_estimators'], 
                     rcv.best_params_['n_estimators'] + 200]
}

In [38]:
print(param_grid)


{'criterion': ['entropy'], 'max_depth': [230], 'max_features': ['auto'], 'min_samples_leaf': [2, 4], 'min_samples_split': [2, 3, 4], 'n_estimators': [300, 400, 600]}


In [39]:
grid_search=GridSearchCV(estimator=RandomForestClassifier(),param_grid=param_grid,cv=10,n_jobs=1,verbose=2)
grid_search.fit(X_train_new,Y_train_resample_flat)


Fitting 10 folds for each of 18 candidates, totalling 180 fits
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.5s
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.6s
[CV] END criterion=entropy, max_depth=230, max_features=auto, min_samples_leaf=2, min_samples_spl

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=1,
             param_grid={'criterion': ['entropy'], 'max_depth': [230],
                         'max_features': ['auto'], 'min_samples_leaf': [2, 4],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [300, 400, 600]},
             verbose=2)

In [40]:
best_grid=grid_search.best_estimator_
best_grid


RandomForestClassifier(criterion='entropy', max_depth=230, min_samples_leaf=2,
                       n_estimators=600)

In [41]:

print_Score(best_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [42]:
import pickle
pickle_out=open('Thyroid_model.pkl','wb')
pickle.dump(grid_search,pickle_out)
pickle_out.close()
